\### Cell 1 – Project Header

---

### Piano MIDI Generator — v0.1

**GOAL**  
Build a small, clean Python tool that can generate simple piano MIDI patterns (melodies and/or chords). This will serve as the foundation for a larger multi-instrument MIDI toolkit.

---

**This project is Stage 1 of my personal MIDI-Tools Roadmap:**

1. Piano Generator (this notebook)  
2. Bass Generator (separate notebook/project)  
3. Drum Generator  
4. Multi-Instrument Generator (combined tracks)  
5. Local App / Ableton Integration (final long-term goal)  

---

**PROJECT PHILOSOPHY**

- I type all “real logic” myself to build skill and intuition.  
- This notebook is both a learning environment and a prototype.  
- All functions will be written cleanly and modular so they can later be moved into one large, polished tool.  

---

**VERSION NOTES**

**v0.1 = core functionality:**

- scale building  
- simple pattern generator  
- pattern → MIDI conversion  
- demo exporter  

**v0.2 (coming next) = GitHub-ready polish:**

- presets  
- humanization  
- high-level API  
- batch generator  
- README builder  

---

_This cell contains documentation only._


### Cell 2 – Plan & Build Checklist

---

This notebook follows a staged roadmap so development stays organized, repeatable, and easy to continue after breaks.

---

### **PHASE 1 — Setup & Basics**

- Choose libraries (pretty_midi, mido)  
- Set up basic MIDI file structure (tempo, time signature)  
- Define a musical scale in a chosen key (e.g., C major, A minor)  

---

### **PHASE 2 — Pattern Logic**

Choose starting patterns:  
- Single-note melodies  
- Simple triad chords  

Then:  
- Write a generator function to produce a short pattern  
- Add randomness:  
  - Note choice  
  - Rhythm placement  
  - Velocity variation  

---

### **PHASE 3 — MIDI Export**

- Convert the generated pattern into PrettyMIDI Note objects  
- Save the result as a `.mid` file  
- Test by dragging into Ableton or any DAW  

---

### **PHASE 4 — Controls & Refinement**

- Add user parameters (bars, BPM, density, scale choice)  
- Clean up function names and folder structure  
- Implement a “main” helper to generate a clip in one call  

---

_This cell contains planning notes only._


Cell 3 – Imports & Library Setup

In [1]:
# Cell 3 - Imports and basic configuration

# Libraries we'll use to build and save Midi files

# Colab-only: install Midi Libraries (safe to re-run in this cell)
!pip install -q pretty_midi mido

import random
from typing import List, Dict

import pretty_midi   # primary library to build and save Midi
import mido         # optional: handy later for low-level Midi inspection/control


# Basic global defaults for the project (can be overriden later)
DEFAULT_TEMPO = 120     # beats per minute (BPM)
DEFAULT_TIME_SIGNATURE = (4, 4) # assume 4/4 for v0.1 and v0.2 core

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 3.2 MB/s eta 0:00:00


Cell 4 – Basic Music Settings & Globals

In [2]:
# Cell 4 - Music settings (key, scale, note range)
# ------------------------------------------------

# Default musical settings (used as fallbacks in generators)
DEFAULT_KEY = 'C'
DEFAULT_SCALE = 'major'

# Piano range we will allow for v0.1 / v0.2 core
# Middle C = 60, C3 = 48, C5 = 72
PIANO_LOW = 48     # C3
PIANO_HIGH = 72    # C5

# Scale interval definitions (in semitones)
SCALE_INTERVALS = {
    'major': [0, 2, 4, 5, 7, 9, 11],
    'minor': [0, 2, 3, 5, 7, 8, 10],
}

# Note name to MIDI-number offset mapping
NOTE_TO_SEMITONE = {
    'C': 0,
    'C#': 1, 'Db': 1,
    'D': 2,
    'D#': 3, 'Eb': 3,
    'E': 4,
    'F': 5,
    'F#': 6, 'Gb': 6,
    'G': 7,
    'G#': 8, 'Ab': 8,
    'A': 9,
    'A#': 10, 'Bb': 10,
    'B': 11,
}



def build_scale(root: str = DEFAULT_KEY, scale_type: str = DEFAULT_SCALE) -> List[int]:
    '''
    Convert a root note (like 'C' or 'A') and a scale type (e.g. 'major' or 'minor')
    into a list of MIDI note numbers within our piano range.
    '''
    # Clean up input
    root = root.strip().capitalize()

    # Validate root note
    if root not in NOTE_TO_SEMITONE:
        raise ValueError(f'Unknown root note: {root}')

    # Validate scale
    if scale_type not in SCALE_INTERVALS:
        raise ValueError(f'Unsupported scale type: {scale_type}')

    # Get semitone offsets for the scale
    scale_offsets = SCALE_INTERVALS[scale_type]
    root_semitone = NOTE_TO_SEMITONE[root]

    # Build list of scale notes within the piano range
    notes: List[int] = []
    for midi_note in range(PIANO_LOW, PIANO_HIGH + 1):
        semitone = midi_note % 12
        interval = (semitone - root_semitone) % 12
        if interval in scale_offsets:
            notes.append(midi_note)

    return notes


Cell 5 – Pattern Representation (Python)


In [3]:
# Cell 5 - Pattern representation


from typing import TypedDict



class NoteEvent(TypedDict):
  time: float
  duration: float
  note: int
  velocity: int

  # (future) can expand with fields like 'humanized' or 'preset_id'


def make_note(time: float, duration: float, note: int, velocity: int = 100) -> NoteEvent:
  '''
  Helper to create a note-event dictionary.
  Used by pattern generators and (later) humanization logic.
  '''
  return {
      'time': time,
      'duration': duration,
      'note': note,
      'velocity': velocity
  }

Cell 6 – Pattern Generator (Melody + Chord Logic)

In [4]:
# Cell 6 - Piano pattern generator (v0.1)



def generate_piano_pattern(
    num_bars: int = 2,
    notes_per_bar: int = 4,
    root: str = DEFAULT_KEY,
    scale_type: str = DEFAULT_SCALE,
    tempo: int = DEFAULT_TEMPO,
) -> List[NoteEvent]:
    '''
    Generate a simple piano pattern by randomly choosing notes from the scale.
    v0.2 will extend this with presets and humanization.
    '''

    if tempo <= 0:
        raise ValueError('Tempo must be greater than zero')



    # Build list of allowed notes in the key/scale
    scale_notes = build_scale(root, scale_type)

    pattern: List[NoteEvent] = []


    # Timing math
    seconds_per_beat = 60.0 / tempo
    beats_per_bar = 4
    beats_per_note = beats_per_bar / notes_per_bar
    seconds_per_note = beats_per_note * seconds_per_beat

    current_time = 0.0

    for bar in range(num_bars):
        for _ in range(notes_per_bar):
                note = random.choice(scale_notes)
                event = make_note(
                    time=current_time,
                    duration=seconds_per_note * 0.95,   # slightly shorter to avoid overlap
                    note=note,
                    velocity=random.randint(80, 110)   # (v0.2) tweak range / humanize later
                )
                pattern.append(event)
                current_time += seconds_per_note

    return pattern


Cell 7 – MIDI Export Stub (Python)


In [5]:
# Cell 7 – Convert pattern to MIDI & save
# ---------------------------------------


def pattern_to_midi(
    pattern: List[NoteEvent],
    filename: str = 'piano_pattern.mid',
    tempo: int = DEFAULT_TEMPO
) -> None:
    '''
    Convert a list of NoteEvent dictionaries into a MIDI file and save it.
    '''
    if not pattern:
        raise ValueError('Pattern is empty; nothing to write to MIDI.')

    # Create a prettyMidi object
    midi = pretty_midi.PrettyMIDI(initial_tempo=tempo)

    # Create a piano instrument (0 = Acoustic Grand Piano)
    piano = pretty_midi.Instrument(program=0, name='Piano')

    # Convert each NoteEvent into a PrettyMIDI Note
    for event in pattern:
        note = pretty_midi.Note(
            velocity=event['velocity'],
            pitch=event['note'],
            start=event['time'],
            end=event['time'] + event['duration'],
        )
        piano.notes.append(note)

    # Add the instrument to MIDI object
    midi.instruments.append(piano)

    # Save the file
    midi.write(filename)

    print(f'Saved MIDI file as: {filename}')



Cell 8 – Quick Test Harness (v0.1 Demo)


In [6]:
# Cell 8 – Quick test runner
# --------------------------


def demo_generate_and_save(
    num_bars: int = 2,
    notes_per_bar: int = 4,
    root: str = DEFAULT_KEY,
    scale_type: str = DEFAULT_SCALE,
    tempo: int = DEFAULT_TEMPO,
    filename: str = 'piano_demo_01.mid',
) -> None:
    '''
    DEMO:
    - Generate a simple pattern
    - Save it to a MIDI file
    '''

    # Generate the pattern
    try:
        pattern = generate_piano_pattern(
            num_bars=num_bars,
            notes_per_bar=notes_per_bar,
            root=root,
            scale_type=scale_type,
            tempo=tempo,
        )
    except Exception as e:
        raise RuntimeError(f'Error generating pattern: {e}')

    print(f'Generated pattern with {len(pattern)} notes.')

    # Convert to MIDI and save
    pattern_to_midi(
        pattern=pattern,
        filename=filename,
        tempo=tempo,
    )


    print('Done! you can now download the MIDI file.')



# Run the demo once
demo_generate_and_save()

Generated pattern with 8 notes.
Saved MIDI file as: piano_demo_01.mid
Done! you can now download the MIDI file.


### Cell 9 – v0.2 Development Overview (Roadmap for Future Work)

---

### **PURPOSE**

This cell is a developer-facing overview of what v0.2 adds on top of v0.1.  
It ensures that every time we reopen the project, we know exactly where we left off and can continue building in a consistent direction.

---

### **SUMMARY**

**v0.1 implemented:**
- Scale builder  
- Piano pattern generator  
- Pattern → MIDI writer  
- Demo function  

**v0.2 will add GitHub-ready polish, including:**
- Clearer structure and defaults  
- Presets system  
- Humanization helpers (velocity & timing)  
- Higher-level API  
- Optional batch/album generator  
- README builder  

---

### **WHAT THE GENERATOR CURRENTLY DOES (v0.1)**

- Builds a musical scale for a chosen KEY and SCALE_TYPE  
- Generates timing, velocity, and note placement  
- Creates a piano-only MIDI clip for a given number of bars  
- Saves the clip as a `.mid` file for easy use in any DAW  

---

### **BAR LENGTH (IMPORTANT CHANGE FOR v0.2)**

- v0.1 default: 2 bars  
- v0.2 default: **4 bars**  
  (Users can still override using `num_bars`)  

---

### **HOW USERS WILL EVENTUALLY USE v0.2**

1. Adjust global defaults in the Config cell:  
   `DEFAULT_KEY`, `DEFAULT_SCALE`, `DEFAULT_NUM_BARS`, `DEFAULT_TEMPO`  

2. Call the high-level API (coming later):  
   `generate_piano_midi(key="A", scale_type="minor", num_bars=4, ...)`  

3. Use optional features:  
   - `PRESETS` for musical styles  
   - `HUMANIZATION` for expressiveness  
   - `BATCH GENERATOR` to create many clips at once  

---

### **CURRENT LIMITATIONS (to be fixed in v0.2)**

- OUTPUT_DIR not implemented yet (files save to working folder)  
- `generate_piano_midi()` not implemented yet  
- No presets or humanization layers implemented yet  
- No batch generator yet  

---

### **NOTE**

This cell is for development guidance only.  
It ensures we continue building v0.2 consistently and do not accidentally take the project in conflicting directions.


Cell 10 – Config & Defaults (Global Settings)

In [7]:
# ============================================================
# Cell 10 – Config & Defaults (Global Settings Overview)
# ============================================================


from pathlib import Path


# Project metadata
PROJECT_NAME = 'piano_midi_generator_v02'

# Where to save generated MIDI files (Colab-friendly default)
OUTPUT_DIR = Path('/content/midi_output')

# Musical defaults for v0.2
# These override earlier defaults from v0.1 when this cell is run.
DEFAULT_KEY = 'C'          # can change to 'A', 'G', ect.
DEFAULT_SCALE = 'major'    # can change to 'minor'
DEFAULT_NUM_BARS = 4       # v0.2 default: 4-bar clips
DEFAULT_TEMPO = 120        # keep consistant with earlier cells

def ensure_output_dir_exists(path: Path = OUTPUT_DIR) -> Path:
    '''
    Ensure that the MIDI output directory exists.
    Returns the Path object so it can be reused.
    '''
    path.mkdir(parents=True, exist_ok=True)
    return path

def make_output_path(filename: str) -> str:
    '''
    Build a full path for a MIDI file inside OUTPUT_DIR.
    Does Not write any files by itself.
    '''

    ensure_output_dir_exists()
    return str(OUTPUT_DIR / filename)




Cell 11 – High-Level API Function

In [8]:
# ============================================================
# Cell 11 – High-Level API Function: generate_piano_midi(...)
# ============================================================
#
# One-call entry point for generating a piano MIDI clip using
# the v0.1 core plus v0.2 config defaults.
#


from typing import Optional

def generate_piano_midi(
    key: Optional[str] = None,
    scale_type: Optional[str] = None,
    num_bars: Optional[int] = None,
    notes_per_bar: int = 4,
    preset_name: Optional[str] = None,        # planned for v0.2 presets
    out_filename: Optional[str] = None,
    apply_humanization: bool = False,         # will be used in Cell 14
    tempo: Optional[int] = None,
) -> str:
    '''
    High-level helper to generate a piano MIDI clip and save it.

    '''

    def clip_patterns_to_bars(pattern, num_bars, beats_per_bar=4.0):
        max_time = num_bars * beats_per_bar
        clipped = []
        for ev in pattern:
            start = ev['time']
            end = start + ev['duration']

            if start >= max_time:
                # Entire event is beyond the clip limit; skip it entirely
                continue

            if end > max_time:
                # trim duration so clip ends exactly at bar boundary
                ev = ev.copy()
                ev['duration'] = max(0.0, max_time - start)

            clipped.append(ev)

        return clipped



    # 1) resolve defaults from config
    key = key or DEFAULT_KEY
    scale_type = scale_type or DEFAULT_SCALE
    num_bars = num_bars or DEFAULT_NUM_BARS
    tempo = tempo or DEFAULT_TEMPO

    # 2) Handle filename and output directory
    if out_filename is None:
        safe_key = str(key).replace('#', 'sharp').replace(' ', '').lower()
        safe_scale = str(scale_type).replace(' ', '').lower()
        base_name = f'piano_{safe_key}_{safe_scale}_{num_bars}bars.mid'
        filepath = make_output_path(base_name)
    else:
        # If filename is supplied, still place it inside OUTPUT_DIR
        filepath = make_output_path(out_filename)

    # 3) Generate the core note pattern using existing v0.1 logic
    pattern = generate_piano_pattern(
        num_bars=num_bars,
        notes_per_bar=notes_per_bar,
        root=key,
        scale_type=scale_type,
        tempo=tempo,

    )

    pattern = clip_patterns_to_bars(pattern, num_bars=num_bars)


    # 4) (FUTURE) apply presets and humanization
    # For now, we just pass through unchanged.

    # 5) Write MIDI
    pattern_to_midi(
        pattern=pattern,
        filename=filepath,
        tempo=tempo,
    )

    print(f'Generated piano MIDI file: {filepath}')
    return filepath







Cell 12 – Usage / Demo Examples

In [9]:
# ============================================================
# Cell 12 – Quick sanity tests for generate_piano_midi(...)
# ============================================================
#
# Run a few sample calls to verify:
# - Defaults work
# - Custom key/scale/num_bars works
# - out_filename is respected
#

print('Test 1 - Defaults (config-driven)...')
midi_path_1 = generate_piano_midi()
print('  Saved MIDI to:', midi_path_1)

print('\nTest 2 - A minor, 8 bars, 4 notes/bar...')
midi_path_2 = generate_piano_midi(
    key='A',
    scale_type='minor',
    num_bars=8,
    notes_per_bar=4,
)
print('  Saved MIDI to:', midi_path_2)

print('\nTest 3 - C major, 4 bars, explicit filename...')
midi_path_3 = generate_piano_midi(
    key='C',
    scale_type='major',
    num_bars=4,
    notes_per_bar=4,
    out_filename='piano_test_custom.mid',
)
print('  Saved MIDI to:', midi_path_3)


Test 1 - Defaults (config-driven)...
Saved MIDI file as: /content/midi_output/piano_c_major_4bars.mid
Generated piano MIDI file: /content/midi_output/piano_c_major_4bars.mid
  Saved MIDI to: /content/midi_output/piano_c_major_4bars.mid

Test 2 - A minor, 8 bars, 4 notes/bar...
Saved MIDI file as: /content/midi_output/piano_a_minor_8bars.mid
Generated piano MIDI file: /content/midi_output/piano_a_minor_8bars.mid
  Saved MIDI to: /content/midi_output/piano_a_minor_8bars.mid

Test 3 - C major, 4 bars, explicit filename...
Saved MIDI file as: /content/midi_output/piano_test_custom.mid
Generated piano MIDI file: /content/midi_output/piano_test_custom.mid
  Saved MIDI to: /content/midi_output/piano_test_custom.mid


Cell 13 – Presets (Patterns & Styles)

In [10]:
# ============================================================
# Cell 13 – PRESETS: Named Musical Patterns & Styles
# ============================================================
#
# A dictionaryof preset musical behaviors. These do not change
# any logic yet (that happens later), but they give structured,
# named configurations for v0.2 pattern generation.
#

PRESETS = {
    'simple_arpeggio': {
        'notes_per_bar': 4,
        'octave_shift': 0,
        'use_chords': False,
        'pattern_type': 'arp',
    },

    'funky_arpeggio': {
        'notes_per_bar': 6,     # more rolling movement
        'octave_shift': 12,     # small jump for energy
        'use_chords': False,
        'pattern_type': 'arp',

    },

    'slow_chords': {
        'notes_per_bar': 2,     # long sustained notes
        'octave_shift': 0,
        'use_chords': True,
        'pattern_type': 'chords',
    },

    'busy_piano': {
        'notes_per_bar': 8,
        'octave_shift': 12,
        'use_chords': True,
        'pattern_type': 'mixed',
    },

}



Cell 14 – Humanization Helpers

In [11]:
# ============================================================
# Cell 14 – Humanization: Velocity & Timing Helpers
# ============================================================
#
# Small helpers to add subtle velocity and timing variation so
# the MIDI feels less robotic and more human.
#

import random
from typing import List, Dict, Any


def humanize_velocities(
    note_events: List[Dict[str, Any]],
    base_velocity: int | None = None,
    spread: int = 8,
    min_velocity: int = 30,
    max_velocity: int = 127,
) -> List[Dict[str, Any]]:
    '''
    Return a new list of note events with slightly varied velocities.

    Each event is expected to have a 'velocity' key. If base_velocity
    is provided, we humanize around that; otherwise we humanize around
    each event's existing velocity.
    '''
    humanized: List[Dict[str, Any]] = []

    for ev in note_events:
        ev_copy = dict(ev)

        base = base_velocity if base_velocity is not None else ev_copy.get('velocity', 90)
        delta = random.randint(-spread, spread)
        new_vel = max(min_velocity, min(max_velocity, base + delta))

        ev_copy['velocity'] = new_vel
        humanized.append(ev_copy)

    return humanized



def humanize_timing(
    note_events: List[Dict[str, Any]],
    max_shift_ticks: int = 8,
    start_key: str = 'start_tick',
) -> List[Dict[str, Any]]:
    '''
    Return a new list of note events with small random timing shifts.


    Each event is expected to have a start-time key [default 'start_tick'].
    We randomly shift by [-max_shift_ticks, max_shift_ticks].
    '''
    humanized: List[Dict[str, Any]] = []

    for ev in note_events:
        ev_copy = dict(ev)

        if start_key in ev_copy:
            shift = random.randint(-max_shift_ticks, max_shift_ticks)
            ev_copy[start_key] = ev_copy[start_key] + shift

        humanized.append(ev_copy)

    return humanized




Cell 15 – Batch Generator (“Album Mode”)

In [12]:
# ============================================================
# Cell 15 – Batch MIDI Generator ("Album Mode")
# ============================================================
# Generate multiple MIDI clips at once using different presets.

def batch_generate_presets(
    key: str,
    scale_type: str,
    num_bars: int,
    preset_names: list[str],
    name_prefix: str = ''
) -> list[str]:
    '''Generate one MIDI file per preset and return list of paths.'''
    output_paths = []

    for preset in preset_names:
        # Build a consistent filename for each preset
        filename = f'{name_prefix}{key}_{scale_type}_{preset}_{num_bars}bars.mid'

        # Reuse the high-level generator
        path = generate_piano_midi(
            key=key,
            scale_type=scale_type,
            num_bars=num_bars,
            preset_name=preset,
            out_filename=filename,
            apply_humanization=False,   # user can change if desired

        )

        output_paths.append(path)

    return output_paths


Cell 16 - One-Click Generate + Download

In [13]:
# ============================================================
# Cell 16 – One-Click Generate + Download (Colab)
# ============================================================
# Simple control panel to generate and download one MIDI clip.

from google.colab import files

def _prompt_or_default(prompt: str, default: str) -> str:
    text = input(f'{prompt} [{default}]: ').strip()
    return text or default

def _prompt_int_or_default(prompt: str, default: int) -> int:
    text = input(f'{prompt} [{default}]: ').strip()
    if not text:
        return default
    try:
        return int(text)
    except ValueError:
        print('Invalid number, using default.')
        return default

print('\n=== One-Click MIDI Generator ===')
key = _prompt_or_default('Key', 'A')
scale_type = _prompt_or_default('Scale type (major/minor)', 'minor')
num_bars = _prompt_int_or_default('Number of bars', 8)
tempo = _prompt_int_or_default('Tempo (BPM)', 128)

print('Generating MIDI clip...')
midi_path = generate_piano_midi(
    key=key,
    scale_type=scale_type,
    num_bars=num_bars,
    tempo=tempo,

)

print('Saved MIDI to:', midi_path)
print('Triggering download...')
files.download(midi_path)


=== One-Click MIDI Generator ===
Key [A]: E
Scale type (major/minor) [minor]: minor
Number of bars [8]: 4
Tempo (BPM) [128]: 128
Generating MIDI clip...
Saved MIDI file as: /content/midi_output/piano_e_minor_4bars.mid
Generated piano MIDI file: /content/midi_output/piano_e_minor_4bars.mid
Saved MIDI to: /content/midi_output/piano_e_minor_4bars.mid
Triggering download...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# 🎹 Piano MIDI Generator — v0.2
A clean, DAW-ready piano MIDI generator written in Python as part of a multi-project MIDI Tools Roadmap.

This notebook is the first project in a series designed to show progression in Python, music-tech engineering, and tool design. Each version adds structure, clarity, and features that build toward a complete multi-instrument MIDI toolkit.

Version 0.2 focuses on:
- Clean project organization
- Reliable MIDI timing
- High-level API
- Preset scaffolding
- Humanization helpers
- Batch generation
- DAW-tested loop correctness

---------------------------------------------------------------------

## 🚀 Overview
This project generates piano MIDI clips in any key or scale using simple musical patterns.

Files save to a Colab-friendly folder and load cleanly into Ableton, FL Studio, Logic, and more.

The notebook is structured into incremental cells, each with a single responsibility (scale building, pattern generation, MIDI export, configuration, presets, etc.).

---------------------------------------------------------------------

## 🧱 Features (v0.2)

### Core Features (from v0.1)
- Builds musical scales from key + scale type
- Generates simple melodic patterns
- Safe piano range (C3–C5)
- Light velocity variation
- Converts patterns to MIDI via PrettyMIDI
- Timing uses seconds (not ticks) for DAW accuracy
- Tested for clean looping

### v0.2 Enhancements
- High-level API: generate_piano_midi(...)
- Automatic filename generation
- Global defaults (DEFAULT_KEY, DEFAULT_SCALE, DEFAULT_NUM_BARS, DEFAULT_TEMPO)
- Output directory system (/content/midi_output)
- Bar-length clipping/enforcement
- PRESETS dictionary for future styles
- Humanization helper functions
- Batch generator for “album” output
- One-click Colab generator with download
- Notebook README for GitHub polish

---------------------------------------------------------------------

## 📂 Project Structure (Cells)

Cell 1  – Project Header  
Cell 2  – Build Checklist  
Cell 3  – Imports & Library Setup  
Cell 4  – Music Settings & Scale Builder  
Cell 5  – Pattern Representation (NoteEvent)  
Cell 6  – Pattern Generator (v0.1 core)  
Cell 7  – Pattern → MIDI Converter  
Cell 8  – Demo Function  
Cell 9  – v0.2 Development Overview  
Cell 10 – Global Config & Paths  
Cell 11 – High-Level API (generate_piano_midi)  
Cell 12 – Usage Examples  
Cell 13 – Preset Scaffold  
Cell 14 – Humanization Helpers  
Cell 15 – Batch Generator  
Cell 16 – One-Click UI (Colab)  
Cell 17 – README (this cell)

Each cell builds on the one before it, following a clean engineering workflow.

---------------------------------------------------------------------

## 📦 Installation

### Local installation:
pip install pretty_midi mido

### Colab installation:
!pip install -q pretty_midi mido

---------------------------------------------------------------------

## ▶️ Quickstart Usage

### 1. Generate a default clip:
midi_path = generate_piano_midi()
print(midi_path)

### 2. Generate a custom clip:
clip = generate_piano_midi(
    key="A",
    scale_type="minor",
    num_bars=8,
    notes_per_bar=4,
    tempo=128
)
print("Saved:", clip)

### 3. Save with a custom filename:
generate_piano_midi(
    key="C",
    scale_type="major",
    num_bars=4,
    out_filename="piano_test.mid",
)

### 4. Batch generation (album mode):
paths = batch_generate_presets(
    key="A",
    scale_type="minor",
    num_bars=8,
    preset_names=["simple_arpeggio", "busy_piano"]
)

### 5. One-click Colab Generator:
Enter key, scale, bars, BPM → auto downloads the MIDI.

---------------------------------------------------------------------

## 🧠 How It Works

### Note event format:
{  
  "time": seconds,  
  "duration": seconds,  
  "note": MIDI number,  
  "velocity": 0–127  
}

### Pattern logic:
- 4 beats per bar  
- Timing based on tempo + notes_per_bar  
- Durations reduced (0.95×) to avoid overlap  
- Exact bar length is enforced  
- DAW tests confirm perfect looping

### API summary:
generate_piano_midi(...) bundles:
- scale building  
- pattern creation  
- bar clipping  
- optional humanization layer  
- MIDI writing  

into a single call.

---------------------------------------------------------------------

## 🔮 Future Work (v0.3+)
- Full preset behaviors (arp modes, chords, octave jumps)
- Humanization tied to tempo
- Swing/groove options
- Chord progression engine
- Pattern styles (broken chords, intervals, arps)
- Multi-track generation (piano + bass + drums)
- Web or CLI interface
- Ableton-style groove templates

---------------------------------------------------------------------

## 👤 Author
Gabe Chavez  
Electronic music producer, DJ, and Python developer  
Building a complete MIDI generator toolkit from scratch.


